In [1]:

import os
import cv2
from  tqdm  import  tqdm 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit
import random
from skimage import feature as ft
from sklearn.decomposition import PCA
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

from xgboost import XGBClassifier
import xgboost
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import NuSVC, SVC
from sklearn.linear_model import SGDClassifier

from sklearn.model_selection import GridSearchCV 
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

Duplicate key in file PosixPath('/Users/sunqian/anaconda3/envs/ICR/lib/python3.10/site-packages/matplotlib/mpl-data/matplotlibrc'), line 789 ('font.family         : sans-serif')
Duplicate key in file PosixPath('/Users/sunqian/anaconda3/envs/ICR/lib/python3.10/site-packages/matplotlib/mpl-data/matplotlibrc'), line 790 ('font.sans-serif     : Microsoft YaHei, SimHei, Arial')


In [3]:
!pip install opencv-python -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.4/35.4 MB 749.0 kB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 759.1 kB/s eta 0:00:0000:0100:01


In [2]:
pip list

Package                   Version
------------------------- ---------
anyio                     3.7.1
appnope                   0.1.3
argon2-cffi               21.3.0
argon2-cffi-bindings      21.2.0
arrow                     1.2.3
asgiref                   3.7.2
asttokens                 2.2.1
async-lru                 2.0.4
attrs                     23.1.0
Babel                     2.12.1
backcall                  0.2.0
backports.zoneinfo        0.2.1
beautifulsoup4            4.12.2
bleach                    6.0.0
certifi                   2023.7.22
cffi                      1.15.1
charset-normalizer        3.2.0
comm                      0.1.3
debugpy                   1.6.7
decorator                 5.1.1
defusedxml                0.7.1
Django                    4.2.3
exceptiongroup            1.1.2
executing                 1.2.0
fastjsonschema            2.18.0
fqdn                      1.5.1
idna                      3.4
image                     1.5.33
importlib-metadata      

In [17]:
!pip install  matplotlib -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 731.9 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.4/232.4 kB 835.0 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 741.3 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 kB 1.3 MB/s eta 0:00:00ta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 3.5 MB/s eta 0:00:00


In [1]:
import cv2
import numpy as np

In [44]:
#以1_Pre_test-h02060.jpg为例
image=cv2.imread('w01870.jpg')

In [45]:
#=============================灰度变换==========================
# 通过将像素值映射到新的范围来增强图像的灰度
min_gray = 0  # 新的最小灰度值
max_gray = 255  # 新的最大灰度值
gray_img_enhanced = cv2.convertScaleAbs(image, alpha=(max_gray-min_gray)/255, beta=min_gray)


In [46]:
cv2.imshow('gray_image_enhanced',gray_img_enhanced)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
#=============================同态滤波==========================
gray = cv2.bilateralFilter(image, 15, 75, 75)
# 对数变换和傅里叶变换
H, W = gray.shape
gray_log = np.log(gray+1)
gray_fft = np.fft.fft2(gray_log)
# 设置同态滤波器参数
c, d, gamma_L, gamma_H, gamma_C = 1, 10, 0.2, 2.5, 1
# 构造同态滤波器
u, v = np.meshgrid(range(W), range(H))
Duv = np.sqrt((u-W/2)**2 + (v-H/2)**2)
Huv = (gamma_H - gamma_L) * (1 - np.exp(-c*(Duv**2)/(d**2))) + gamma_L
Huv = Huv * (1 - gamma_C) + gamma_C
# 进行频域滤波
gray_fft_filtered = Huv * gray_fft
gray_filtered = np.fft.ifft2(gray_fft_filtered)
gray_filtered = np.exp(np.real(gray_filtered))-1
# 转为uint8类型
gray_filtered = cv2.normalize(gray_filtered, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)


In [47]:
#=============================对比拉伸==========================
norm_img1 = cv2.normalize(image,None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
# scale to uint8
norm_img1 = (255 * norm_img1).astype(np.uint8)


In [48]:
cv2.imshow('norm_image1',norm_img1)
cv2.waitKey(0) 
cv2.destroyAllWindows()

In [49]:
#=============================对数变换==========================
# Apply log transformation method
c = 255 / np.log(1 + np.max(image))
log_image = c * (np.log(image + 1))
# Specify the data type so that
# float value will be converted to int
log_image = np.array(log_image, dtype=np.uint8)


/var/folders/yy/xl5jyrln4lddx85rv3db9s_r0000gn/T/ipykernel_1434/1460892277.py:4: RuntimeWarning: divide by zero encountered in log
  log_image = c * (np.log(image + 1))
/var/folders/yy/xl5jyrln4lddx85rv3db9s_r0000gn/T/ipykernel_1434/1460892277.py:7: RuntimeWarning: invalid value encountered in cast
  log_image = np.array(log_image, dtype=np.uint8)


In [50]:
cv2.imshow('log_image',log_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [51]:
#=============================幂律变换（伽马变换）==========================
# Apply Gamma=0.4 on the normalised image and then multiply by scaling constant (For 8 bit, c=255)
gamma_point_eight = np.array(255 * (image / 255) ** 0.8, dtype='uint8')


In [52]:
cv2.imshow('gamma_point_eight_image',gamma_point_eight)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [53]:
# 图像均衡化
def equalize(image):
    # 分割B,G,R （cv2读取图像的格式即为[B,G,R]，与matplotlib的[R,G,B]不同）
    b,g,r = cv2.split(image)
    # 依次均衡化
    b = cv2.equalizeHist(b)
    g = cv2.equalizeHist(g)
    r = cv2.equalizeHist(r)
    # 结合成一个图像
    equ_img = cv2.merge((b,g,r))
    
    return equ_img


In [54]:
equ_img=equalize(image)

In [55]:
cv2.imshow('equ_image',equ_img)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [56]:
#  二值化、过滤滤波
def extractGreen(image):
    # 绿色范围
    lower_green = np.array([0, 0, 0], dtype="uint8")  # 颜色下限
    upper_green = np.array([250, 250, 250], dtype="uint8")  # 颜色上限
    
    # 高斯滤波
    img_blur = cv2.GaussianBlur(image, (3, 3), 0)
    img_blur = cv2.cvtColor(img_blur, cv2.COLOR_BGR2HSV)
    # 根据阈值找到对应颜色，二值化
    mask = cv2.inRange(img_blur, lower_green, upper_green)
    
    # 掩膜函数
    output = cv2.bitwise_and(image, image, mask=mask)

    return output


In [57]:
masked_img=extractGreen(image)

In [58]:
cv2.imshow('masked_image',masked_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [39]:
import skimage.filters.rank as sfr
from skimage.morphology import disk

In [59]:
#灰度增强  效果最好
import cv2
from skimage import data,color
import matplotlib.pyplot as plt
from skimage.morphology import disk
import skimage.filters.rank as sfr
cv2.imshow('original',image)
grayImage=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
auto=sfr.enhance_contrast(grayImage,disk(5))#半径为5的圆形滤波器
cv2.imshow("result",auto)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [61]:
#特征提取
import cv2
 
# 初始化SIFT检测器
sift = cv2.SIFT_create()
 
# 使用detectAndCompute方法检测关键点并计算描述符
keypoints, descriptors = sift.detectAndCompute(image, None)
 
# 绘制关键点（可选）
output_image = cv2.drawKeypoints(image, keypoints, None)
 
# 显示图像
cv2.imshow('Output Image', output_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [42]:
#resize
newimg=cv2.resize(image, (128, 128), interpolation=cv2.INTER_CUBIC)
print(np.shape(newimg))

(128, 128, 3)


In [43]:
#特征提取
import cv2
 
# 初始化SIFT检测器
sift = cv2.SIFT_create()
 
# 使用detectAndCompute方法检测关键点并计算描述符
keypoints, descriptors = sift.detectAndCompute(newimg, None)
 
# 绘制关键点（可选）
newimg_output_image = cv2.drawKeypoints(newimg, keypoints, None)
 
# 显示图像
cv2.imshow('newimg_output_image', newimg_output_image)
cv2.waitKey(0)
cv2.destroyAllWindows()